<a href="https://colab.research.google.com/github/Majmejia/Business-Inteligence/blob/main/Reto_2_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Se Procede a cargar las librerias

In [ ]:
#Librerias cientificas
import numpy as np
import pandas as pd

#Librerias Árbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

1. Se procede a cargar los datos

In [ ]:
nxl='/content/2. CollegeAdmisions_Int_M. Reto 2.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0)
XDB.head(100)
XDB=XDB.dropna() #Elimina filas vacias o datos erroneos

#Seleccionar las variables
XD=XDB[['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']] #Variables de entrada
yd=XDB[['Admitted']] #Variables de salida

#Se convierte la variable 'Gender' En una variable categorica
XD.loc[:,'Gender'] = XD['Gender'].map({'F': 0, 'M': 1})


,Edu_Parent1,Edu_Parent2,Gender,White,Asian,HSGPA,SAT/ACT
6,6,6,0,1,0,4.26,1430
28,7,6,1,1,0,3.95,1340
46,3,6,1,1,0,4.33,1200
61,7,7,0,1,0,4.27,1530
65,6,7,0,0,0,4.04,1270
...,...,...,...,...,...,...,...
17330,2,5,1,0,0,2.80,1010
17332,7,4,1,1,0,3.04,1130
17333,4,6,1,1,0,2.80,1210
17334,7,6,1,0,0,3.42,1000


**Descripción de los Datos**
De acuerdo con la base de datos de "College Admisions", esta posee un total de 1528, divididos entre quienes fueron admitidos y negados a la universidad. Todos estos aspirantes se dividen en un total de 8 variables.

**Descripción**

*Edu_Parent1 y Edu_Parent2: Hacen referencia al nivel educativo del padre y madre del aspirante

*Gender: Indica el genero (Hombre o Mujer) del aspirante.

*White: Indica si el aspirante es blanco (1) o caso contrario (0)

*Asian:Indica si la persona es de presedencia asiatica o no.

*HSGPA:Indica el rendimiento académico del aspirante durante la escuela secundaria y se utiliza comúnmente en el proceso de admisión a la universidad.

*SAT/ACT: Puntuación obtenida por el aspirante en una de las dos pruebas estandarizadas utilizadas para la admisión a la universidad en los Estados Unidos

*Admitted: Esta variable determina si un estudiante fue admitido o no a la universidad


2. Se procede con la implementación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4, random_state=42)
mar.fit(XD,yd)#Para ajustar el modelo del arbol entre las variables de entrada y salida

#Se procede con la grafica
from pydotplus import graph_from_dot_data #Me crea la grafica
from sklearn.tree import export_graphviz #Me lo exporta a PNG, JPEG, TIFF

ve=['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT'] #Nombres para el árbol
dot_data=export_graphviz(mar,feature_names=ve) #Crear el gráfico
graph=graph_from_dot_data(dot_data)
graph.write_png('ArbolDesicion2.png')



True

3. Se procede con el pronostico de la admisión para las personas agrupadas en la base de datos de pronóstico.

In [ ]:
# Count admitted students
admitted_count = XDB['Admitted'].value_counts()['Yes']

# Count not admitted students
not_admitted_count = XDB['Admitted'].value_counts()['No']

# Print the results
print("Admitted students:", admitted_count)
print("Not admitted students:", not_admitted_count)

Admitted students: 1303
Not admitted students: 225


4. Se procede a determinar el número de nodos puros para el árbol asociado con la estructura del modelo.

In [ ]:
# Se obtiene la estructura del Árbol
tree = mar.tree_

# Se comienza con el conteo de nodos puros
pure_nodes_count = 0

# Se analiza a traves de todos los nodos del Árbol
for node_id in range(tree.node_count):
    # Check if the node is a leaf node (no children)
    if tree.children_left[node_id] == tree.children_right[node_id]:
        # Check if the node is pure (all samples belong to the same class)
        if tree.impurity[node_id] == 0:
            pure_nodes_count += 1

# Copiar el numero de nodos puros
print("Number of pure nodes:", pure_nodes_count)

Number of pure nodes: 7


5. Llevar a cabo el pronóstico de la admisión para las personas agrupadas en la base de datos de pronóstico.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# ... (tu código para crear y entrenar el modelo 'mar')

# Cargar datos de la hoja 1 del Excel
nxl ='/content/2. CollegeAdmisions_Int_M. Reto 2.xlsx'
XDBO = pd.read_excel(nxl, sheet_name=1) # Asumiendo que la hoja 1 se llama 'Hoja1'

# Seleccionar las variables predictoras
XD0 = XDBO[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]

# Convertir 'Gender' a numérica en XD0
XD0['Gender'] = XD0['Gender'].map({'F': 0, 'M': 1})

# Realizar la predicción
y_pred = mar.predict(XD0)

# Agregar las predicciones al DataFrame original
XDBO['Admisión_Predicha'] = y_pred

# Guardar el DataFrame con las predicciones en un nuevo archivo Excel
XDBO.to_excel("Admisiones_con_Predicciones.xlsx", index=False)

<ipython-input-36-2a35c1d781a0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XD0['Gender'] = XD0['Gender'].map({'F': 0, 'M': 1})


6. Evaluar el comportamiento del modelo


In [ ]:
ydp=mar.predict(XD) #Prónostico sobre la base de datos original
cm=confusion_matrix(yd,ydp)
print("La matriz de confusión es:\n",cm)

VN=cm[0,0] #Verdaderos Negativos
VP=cm[1,1] #Verdaderos Positivos
FP=cm[0,1] #Falsos Positivos
FN=cm[1,0] #Falsos Negativos

Exactitud=(VP+VN)/(VP+VN+FP+FN)
Sensibilidad=VP/(VP+FN)
Especificidad=VN/(VN+FP)
TasaError=(FP+FN)/(VP+VN+FP+FN)
PredNeg=VN/(VN+FP)
Precision=VP/(VP+FP)
PredNeg=VN/(VN+FN)

print("La tasa de falsos negativos del modelo es:\n",PredNeg)
print("La precisión del modelo es:\n",Precision)
print("Tasa de Error=",TasaError)
print("Predicción Negativa=",PredNeg)
print("Exactitud=",Exactitud)
print("Sensibilidad=",Sensibilidad)
print("Especificidad=",Especificidad)

La matriz de confusión es:
 [[  20  205]
 [   2 1301]]
La tasa de falsos negativos del modelo es:
 0.9090909090909091
La precisión del modelo es:
 0.8638778220451527
Tasa de Error= 0.1354712041884817
Predicción Negativa= 0.9090909090909091
Exactitud= 0.8645287958115183
Sensibilidad= 0.9984650805832693
Especificidad= 0.08888888888888889


**Análisis de los resultados**

La matriz de confusión muestra una predominancia por los valores verdaderos positivos (1301). Esto hizo que el modelo alcanzara valores por encima del 80% en promedio para los indices de: precisión del modelo, predicción negativa, exactitud y sesibilidad. Lo que demuestra un buen funcionamiento del modelo para determinar a los aspirantes que fueron admitidos.

La base de datos no muestra ningun campo sin decisiones tomadas en cuando la admision de los estudiantes. Se destaca que el numero de personas admitidas fue mucho mayor al de los no admitidos (1303 admitidos y 225 no admitidos).De acuerdo con los indices el modelo muestra un mejor comportamiento frente a la sensibilidad (0.998) o la predicción negativa de los datos (0.909).